# 0 Functions

In [2]:
%run "0k Benchmark Imports CMZ.ipynb"

/home/ben/miniconda3/bin/python
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
astropy 5.1.dev153+gb740594dc
reproject 0.8
spectral_cube 0.6.1.dev22+g003ef16
/home/ben/.local/lib/python3.8/site-packages/spectral_cube/__init__.py
/home/ben/.local/lib/python3.8/site-packages/astrodendro/__init__.py
1.23.1 Numpy


# Part 8: Dendograms

In [4]:

#CO 3-2 all resses starting at 4.3


Num=5
Overlaps=0#4
Ram_Limiter=1#What percent of the cube my ram can handle
LineN="CMZ_CO_J3_2_all_NEW"
Name = "CMZ_CO_J3_2_all_NEW"
name = "CMZ_CO_J3_2_CM_Contours_all_NEW.jpeg"

Abs_Levels = ["All","None","No Clusters", "None (m)", "None All Channels"]



iterations = 6
iter_factor = 1/5
#Line_Name = '_CMZ_CO_J3_2_4.3_start'

NM= False

Smoothe_4 = False
Match_to_HCO = False
if(Smoothe_4):
    Smoothe_title="G_width_4"
else:
    Smoothe_title=''
    if(Match_to_HCO):
        Smoothe_title='Smoothe_to_3.3'
Smoothe_2_5 =True
if(Smoothe_2_5):
    Smoothe_title="Smoothe_to_2.5"

Edge_cases=True

if(Edge_cases):
    Edge_title="Edges"
else:
    Edge_title=''
    
Line_Name = '_CMZ_CO_J3_2_4.3_start'

if(NM):
    Line_Name_Save = Line_Name+'_NM'+Smoothe_title+Edge_title
else:
    Line_Name_Save=Line_Name+Smoothe_title+Edge_title



ovs = 3 #over sample factor for beam
min_vel = 3.3#km/s
FOV = [70,360]#pc
Min_res=4.3*u.pc#Smallest resolution
Min_beam_req = 1/5

Params = [iterations,iter_factor,ovs,min_vel,FOV,Min_res,Min_beam_req]
savePath='/home/ben/Documents/Grad Stuff/MM data/Result Files'
if(NM):
    
    Name_File = "CMZ_Names_New_All_Kinds_NM"+Smoothe_title+Edge_title
else:
    
    Name_File= "CMZ_Names_New_All_Kinds"+Smoothe_title+Edge_title

np.savetxt(Line_Name+"_Params", Params,fmt='%s')


cen_p1 = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')
cen_p2 = SkyCoord('-00d03m20.76s  ', '-00d02m46.176s', frame='galactic')



In [3]:
savePath='./Result Files'



In [ ]:
#Krieger

#Beam
Abs_Level = Abs_Levels[0]



if gal =="GC":
    pathCont = '4.3pc_beam_CMZ_850um_Cont_140x800pc.fits'#'CMZ_Continuum_Splice.fits'
else:
    pathCont = 'Continuum_Reproject.fits'





start_num=0
km=0
for kl in range(5,6):
      
    Prime_Beam = Min_res*((kl*iter_factor+1))#
    max_size = Prime_Beam*7   
    if(kl==5):
        Prime_Beam = 3*u.pc#Nico
        km=5
        FOVp=[200,800]
        max_size = 20#pc
        if(Match_to_HCO):
            Prime_Beam = 4.3*u.pc#Nico
            FOVp=[70,360]

    else:
        FOVp=FOV
    print(Prime_Beam)
    
    
    vel_prime = min_vel*((km*iter_factor+1))

    if(km==5):
        vel_prime=2.5
        ovs=5
        if(Match_to_HCO):
            vel_prime=3.3
            ovs=3
    if(kl==5,km==5):
        Trunks=True
    else:
        Trunks=True




    try:
        Names=list(np.load(Name_File+".npy",allow_pickle=True))
        if(iterations!=np.shape(Names)[3]):
            del popasasd
    except Exception as e:

        print(e)
        print("initialized names")
        Names = list(np.empty((20,iterations,iterations,iterations,iterations,iterations,iterations),dtype=object))


    #Continuum image
    scCont = spectral_cube.Projection.from_hdu(fits.open(pathCont)[0])
    #Put this up here for the column density map
    metadata = {}
    metadata["distance"] = dist_cmz
    arc_per_pix_yc =  abs(scCont.hdu.header["CDELT1"]*3600.0 * u.arcsec)/u.pix
    arc_per_pix_xc =  abs(scCont.hdu.header["CDELT2"]*3600.0 * u.arcsec)/u.pix
    beam_majorc =  scCont.hdu.header["BMAJ"]*3600.0 * u.arcsec
    beam_minorc =  scCont.hdu.header["BMIN"]*3600.0 * u.arcsec
    beam_area_ratioc = beam_minorc*beam_majorc/arc_per_pix_yc/arc_per_pix_xc*1.13309#beam_area_ratioc = beam_minorc*beam_majorc/arc_per_pix_yc/arc_per_pix_xc#This is for FWHM, use *(2*np.sqrt(2*np.log(2)))**2#For gaussian beam
    metadata['beam_area_ratioc']=beam_area_ratioc
    pc_per_pixelc = abs(scCont.hdu.header["CDELT1"])/180*np.pi*metadata['distance']/u.Mpc
    print(pc_per_pixelc,"MPC per pix")

    #Make subcube


    if(NM):
        Cube_Name_Load = 'Noise_Matched_'+"Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'
    else:
        Cube_Name_Load = "Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_NEW'+Smoothe_title+'.fits'
    path = Cube_Name_Load

    Qp = SpectralCube.read(path).with_spectral_unit(u.km/u.s,velocity_convention="radio") 
    Qp.allow_huge_operations=True

    Q = Qp.to(u.K)#Jy to Kelvin

    sc = Q.unmasked_copy()



    scW = sc.wcs[:][:][0]
    dat = sc.hdu.data[int(len(sc)*((1-Ram_Limiter)/2)):int(len(sc)*(Ram_Limiter + (1-Ram_Limiter)/2)),:,:]
    scn = sc[int(len(sc)*((1-Ram_Limiter)/2)):int(len(sc)*(Ram_Limiter + (1-Ram_Limiter)/2)),:,:]
    scF = scn
    datn = dat


    #m=.115

    #print(m,"Presumed Noise (K)")

    #Continuum image
    scCont = spectral_cube.Projection.from_hdu(fits.open(pathCont)[0])
    #Do the same thing to the continuum image
    scCont.allow_huge_operations=True
    scContW = scCont.reproject(scF.moment0().header)
    #m=.115


    scW = sc.wcs[:][:][0]




    #Do the same thing to the continuum image

    Continuum_Data  =scContW.hdu.data
    scWCS = scF.wcs[:][:][0]

    moment_0_sub  = scF.to(u.K).moment(order=0,how='slice')            # Calculate the Moment 0 map 



    ######Moment 0 for both
    ######and cont
    Make_Plot((LineN+" Moment 0"),"Moment 0 (K km/s)",moment_0_sub.hdu.data,0,np.nanmax(moment_0_sub.hdu.data),moment_0_sub.wcs,1,1,1,True)


    cSD = (Flux_to_Mass(scContW.hdu.data*u.Jy/u.beam/beam_area_ratioc*u.pix**2*u.beam)/(pc_per_pixelc*10**6*3.086*10**18*u.cm)**2)*(1.989*10**30*u.kg/u.M_sun)*Num_per_kg/u.kg #Flux goes to luminosity for a gaussian beam and goes to column density for square pixels, mass goes to kg to number





    rm=moment_0_sub.hdu.data/cSD
    rmU = rm*u.K*u.km/u.s#Just put back in the units
    rmU = np.array(rmU /(u.K*u.km/u.s)/u.cm**2*10**22,dtype='float64')#Now remove them all
    print(rmU)

    ######ratio

    bp = np.where(cSD<=.0001*10**22/u.cm**2)
    bp2 = np.where( moment_0_sub.hdu.data < .00022)
    Continuum_Data[bp] = np.nan
    cSD[bp]=np.nan
    rmU[bp]=np.nan
    rmU[bp2]=np.nan

    Non_nan=((datn[0,:,0:int(np.shape(datn)[2]/2)]>0)  | (datn[0,:,0:int(np.shape(datn)[2]/2)]<0 ))

    m = (np.nanstd(datn[0,:,0:int(np.shape(datn)[2]/2)],where= Non_nan)) #Noise K
    mp=m
    if(Abs_Level=="All"):
        pass
    if(Abs_Level=="None"):
        datn[np.where(datn<0)]=np.nan
    if(Abs_Level=="None (m)"):
        datn[np.where(datn<-m)]=np.nan
    if(Abs_Level=="None All Channels"):
        for lmi in range(len(datn)):
            bpP = np.where(datn[lmi]<-m)
            for lmj in range(len(datn)):
                datn[lmj][bpP]=np.nan
    if(Abs_Level=="No Clusters"):

        #IDs,RAs,Decs,HWHM  =    np.genfromtxt("NGC_Clusters_"+"IDs"+"",dtype=type("12h23.14s")),np.genfromtxt("NGC_Clusters_"+"RAs"+"",dtype=type("12h23.14s")),np.genfromtxt("NGC_Clusters_"+"Decs"+"",dtype=type("12h23.14s")),np.genfromtxt("NGC_Clusters_"+"HWHM"+"")

        HWHM_rad,CDs_New,glons_New,glats_New,IDs_New   =    np.genfromtxt("CMZ_Clusters_"+"HWHM"+""),np.genfromtxt("CMZ_Clusters_"+"CD"+""),np.genfromtxt("CMZ_Clusters_"+"Glon"+""),np.genfromtxt("CMZ_Clusters_"+"Glat"+""),np.genfromtxt("CMZ_Clusters_"+"IDs"+"")

        datn=  Mask_Clusters_CMZ(HWHM_rad,scWCS,scF.hdu.header,datn,glons_New,glats_New,One_Pc=False,One_Pc_Size=1,HWHM_Fac=3)

    print(m,"Unmatched Noise (K)", "Abs = ",Abs_Level)


    Make_Plot("Column Density","(#/cm^2)",cSD,float(np.nanmin(cSD*u.cm**2)),float(np.nanmax(cSD*u.cm**2)),scCont.wcs,1,1,1,True)
    Make_Plot((LineN+" Moment 0/Column Density"),"Moment 0 over Column Density of the Continuum (K km/s/(#/cm^2))",rmU,np.nanmean(rmU)*.5,abs(np.nanmean(rmU))*8,scCont.wcs,1,1,1,True)

    print(np.nanmin(rmU))



    header = scF.hdu.header
    print()
    #make metadata for the dendrogram

    try:
        freq = header["RESTFREQ"]*u.Hz#
        metadata['wavelength']=299792458*u.m/header["RESTFREQ"]#
        print(1,freq,metadata['wavelength'])
    except:
        freq = header["RESTFRQ"]*u.Hz#
        metadata['wavelength']=299792458*u.m/header["RESTFRQ"]#
    metadata['data_unit'] =scF[0][0][0].unit# header['BUNIT']
    metadata['arc_per_pix_y'] =  abs(header["CDELT1"]*3600.0 * u.arcsec)/u.pix
    metadata['arc_per_pix_x'] =  abs(header["CDELT2"]*3600.0 * u.arcsec)/u.pix


    metadata['beam_major'] =  header["BMAJ"]*3600.0 * u.arcsec
    metadata['beam_minor'] =  header["BMIN"]*3600.0 * u.arcsec
    beam_area_ratio = metadata['beam_minor']*metadata['beam_major']/metadata['arc_per_pix_y']/metadata['arc_per_pix_x']*1.13309#beam_area_ratio = metadata['beam_minor']*metadata['beam_major']/metadata['arc_per_pix_y']/metadata['arc_per_pix_x']#This is for FWHM, use *(2*np.sqrt(2*np.log(2)))**2#For gaussian beam
    metadata['beam_area_ratio']=beam_area_ratio
    metadata['spatial_scale'] = np.sqrt(abs(header["CDELT1"])*u.degree**2*abs(header["CDELT2"]))
    #area_res = abs(header["CDELT1"])*abs(header["CDELT2"])*(np.pi/180*3.5)**2#mpc^2/pix^2
    #print(area_res,type(area_res))

    print(metadata['beam_minor'],metadata['beam_major'])
    print(beam_area_ratio)
    #metadata["wcs"] = wcs
    metadata["velocity_scale"] = abs(header["CDELT3"])*u.km/u.s#u.km/u.s

    metadata["vaxis"]=0
    metadata["wcsu"]=scF.wcs


    #for k3 in range(start_num):
    k3=2


    beam_req = Min_beam_req*(k3+1)
    if kl==5:
        beam_req=.9/3
    k4=0
    print(kl,km,k3,k4)
    pix_thresh_factor = k4+1
    k6=2
    delt_factor = (3/5+k6/5)

    k7=2
    noise_factor = (3/5+k7/5)
    if(kl==5):
        m=.115
    else:
        m=mp
    print(kl,km,k3,k4,k6,k7)
    print("actiev noise = ",m)
    try:



        Cube_Name_Save = Name+"_Cropped_"+str(Prime_Beam.value)+"pc_beam_"+Line_Name_Save+str(FOVp[0])+"x"+str(FOVp[1])+'pc_'+str(vel_prime)+'_vel_res_'+str(beam_req)+"_beam_size_req_"+str(pix_thresh_factor)+'_times_beam_pixels_'+str(delt_factor)+"_delt_factor_"+str(noise_factor*5)+"_SNR_NEW.fits"



        Names[Num][kl][km][k3][k4][k6][k7] = Cube_Name_Save

        NameS = (Cube_Name_Save+"Sigmas")
        NameR = (Cube_Name_Save+"Radii")
        NameCol = (Cube_Name_Save+"_Column")
        NameLum = (Cube_Name_Save+"_Luminosities")

        NameFlux = (Cube_Name_Save+"Flux_Dense")
        NameRFF = (Cube_Name_Save+"Rad_For_Flux")

        print(Cube_Name_Save)

        

        np.save(Name_File,Names)




        d = astrodendro.Dendrogram.compute(datn,min_delta=m*delt_factor,min_value=m*5*(noise_factor),min_npix=beam_area_ratio.value*pix_thresh_factor) #The main culprit
        d.save_to((Cube_Name_Save+"_Dendrogram"),"fits")
        
        except Exception as e:
            print(e)
            print(kl,km,k3,k4)
            print("Failed")
            print("-"*60)
            traceback.print_exc(file=sys.stdout)

